# 쿠팡 데이터 크롤링

## 인형, 장난감
1. 제품명
2. 성별
3. 가격
4. url
5. img
6. 연령대

## 기저귀
1. 제품명
2. 성별
3. 브랜드
4. 사이즈
5. 가격
6. url
7. img

___

## Global functions, parameters

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
def getURL(seed, params=None, select='a', num_retries=2):
    html = requests.get(seed, params=params, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getURL(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    links = dom.select(select)
    
    return ['http://www.coupang.com' + link['href'] for link in links if link.has_attr('href')]

In [3]:
def getItemNum(url, params, num_retries=2):
    params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
    html = requests.get(url, params=params_str, headers=headers)
    print(html.url)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getItemNum(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    result = dom.select('em.newcx-product-total-count')
    
    return result

In [4]:
headers = {
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    "accept-encoding":"false"
}

___

### 과자
1. 제품명
2. 브랜드
3. 가격(int)
4. url
5. img
6. 연령대

In [5]:
month_filter_snack = {
    '3개월 이하': '221915%23attr_11932%2413989%40DEFAULT',
    '3개월 이상 7개월 이하': '221915%23attr_11932%2413990%40DEFAULT',
    '7개월 이상 11개월 이하': '221915%23attr_11932%2413991%40DEFAULT',
    '11개월 이상 23개월 이하': '221915%23attr_11932%2413992%40DEFAULT',
    '23개월 이상 35개월 이하': '221915%23attr_11932%2413993%40DEFAULT',
    '35개월 이상': '221915%23attr_11932%2413994%40DEFAULT'
}

In [6]:
monthList_snack = ['3개월 이하', '3개월 이상 7개월 이하', '7개월 이상 11개월 이하', '11개월 이상 23개월 이하', '23개월 이상 35개월 이하', '35개월 이상']

In [7]:
params_snack = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "221915",
    "rating": "0",
}

In [8]:
seed_snack = 'http://www.coupang.com/np/categories/222015'

In [11]:
result_snack = {
    monthList_snack[0]: None,
    monthList_snack[1]: None,
    monthList_snack[2]: None,
    monthList_snack[3]: None,
    monthList_snack[4]: None,
    monthList_snack[5]: None
}

for i in range(6):
    currentpage = 1
    params_snack['filter'] = month_filter_snack[monthList_snack[i]]
    temp = []
    
    while True:
        params_snack['page'] = str(currentpage)
        print(params_snack['filter'])
        if len(getItemNum(seed_snack, params_snack)) == 0:
            break
        print(params_snack['page'])
        temp.extend(getURL(seed_snack, params_snack, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_snack[monthList_snack[i]] = temp

221915%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/222015?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221915%23attr_11932%2413989%40DEFAULT&component=221915&rating=0
1
221915%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/222015?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221915%23attr_11932%2413989%40DEFAULT&component=221915&rating=0
2
221915%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/222015?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221915%23attr_11932%2413989%40DEFAULT&component=221915&rating=0
3
221915%23attr_11932%2413989%40DEFAULT
http://www.coupan

In [12]:
for (k, v) in result_snack.items():
    print(k, len(v))

3개월 이하 180
3개월 이상 7개월 이하 420
7개월 이상 11개월 이하 300
11개월 이상 23개월 이하 180
23개월 이상 35개월 이하 60
35개월 이상 60


In [27]:
snackList = []

In [46]:
def getInfo_snack(url, monthInfo, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_snack(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0:
        brand = dom.select('a.prod-brand-name')[0].get_text()
        name = dom.select('h2.prod-buy-header__title')[0].get_text()
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        url = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        snackList.append(tuple([brand, name, price, url, img, monthInfo]))
    
    return None

In [28]:
for i in range(6):
    for url in result_snack[monthList_snack[i]]:
        getInfo_snack(url, monthList_snack[i])

In [29]:
len(snackList)

1190

___

### 분유
1. 제품명
2. 브랜드
3. 가격(int)
4. url
5. img
6. 연령대

In [20]:
month_filter_milkPowder = {
    '3개월 이하': '221895%23attr_11932%2413989%40DEFAULT',
    '3개월 이상 7개월 이하': '221895%23attr_11932%2413990%40DEFAULT',
    '7개월 이상 11개월 이하': '221895%23attr_11932%2413991%40DEFAULT',
    '11개월 이상 23개월 이하': '221895%23attr_11932%2413992%40DEFAULT',
    '23개월 이상 35개월 이하': '221895%23attr_11932%2413993%40DEFAULT',
    '35개월 이상': '221895%23attr_11932%2413994%40DEFAULT'
}

In [21]:
monthList_milkPowder = ['3개월 이하', '3개월 이상 7개월 이하', '7개월 이상 11개월 이하', '11개월 이상 23개월 이하', '23개월 이상 35개월 이하', '35개월 이상']

In [18]:
params_milkPowder = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "221895",
    "rating": "0",
}

In [37]:
seed_milkPowder = 'http://www.coupang.com/np/categories/221995'

In [22]:
result_milkPowder = {
    monthList_milkPowder[0]: None,
    monthList_milkPowder[1]: None,
    monthList_milkPowder[2]: None,
    monthList_milkPowder[3]: None,
    monthList_milkPowder[4]: None,
    monthList_milkPowder[5]: None
}

for i in range(6):
    currentpage = 1
    params_milkPowder['filter'] = month_filter_milkPowder[monthList_milkPowder[i]]
    temp = []
    
    while True:
        params_milkPowder['page'] = str(currentpage)
        print(params_milkPowder['filter'])
        if len(getItemNum(seed_milkPowder, params_milkPowder)) == 0:
            break
        print(params_milkPowder['page'])
        temp.extend(getURL(seed_milkPowder, params_milkPowder, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_milkPowder[monthList_milkPowder[i]] = temp

221895%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413989%40DEFAULT&component=221895&rating=0
1
221895%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413989%40DEFAULT&component=221895&rating=0
2
221895%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413989%40DEFAULT&component=221895&rating=0
3
221895%23attr_11932%2413989%40DEFAULT
http://www.coupan

http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=6&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413992%40DEFAULT&component=221895&rating=0
221895%23attr_11932%2413993%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413993%40DEFAULT&component=221895&rating=0
1
221895%23attr_11932%2413993%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413993%40DEFAULT&component=221895&rating=0
221895%23attr_11932%2413994%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&bra

In [24]:
for (k, v) in result_milkPowder.items():
    print(k, len(v))

3개월 이하 540
3개월 이상 7개월 이하 480
7개월 이상 11개월 이하 60
11개월 이상 23개월 이하 300
23개월 이상 35개월 이하 60
35개월 이상 60


In [25]:
milkPowderList = []

In [47]:
def getInfo_milkPowder(url, monthInfo, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_milkPowder(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0:
        brand = dom.select('a.prod-brand-name')[0].get_text()
        name = dom.select('h2.prod-buy-header__title')[0].get_text()
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        url = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        milkPowderList.append(tuple([brand, name, price, url, img, monthInfo]))
    
    return None

In [30]:
for i in range(6):
    for url in result_milkPowder[monthList_milkPowder[i]]:
        getInfo_milkPowder(url, monthList_milkPowder[i])

In [31]:
len(milkPowderList)

1487

___

### 의류
1. 제품명
2. 성별
3. 가격
4. url
5. img
6. 연령대

In [58]:
gender_list = ['여아', '남아', '남녀공용']

In [59]:
month_filter_clothes_girl = {
    '0~3개월': '223745%23attr_11872%24150%40DEFAULT',
    '3~6개월': '223745%23attr_11872%24151%40DEFAULT',
    '6~9개월': '223745%23attr_11872%24152%40DEFAULT',
    '9~12개월': '223745%23attr_11872%24153%40DEFAULT',
    '12~18개월': '223745%23attr_11872%24154%40DEFAULT',
    '18~24개월': '223745%23attr_11872%24155%40DEFAULT'
}

In [60]:
month_filter_clothes_boy = {
    '0~3개월': '223746%23attr_11872%24150%40DEFAULT',
    '3~6개월': '223746%23attr_11872%24151%40DEFAULT',
    '6~9개월': '223746%23attr_11872%24152%40DEFAULT',
    '9~12개월': '223746%23attr_11872%24153%40DEFAULT',
    '12~18개월': '223746%23attr_11872%24154%40DEFAULT',
    '18~24개월': '223746%23attr_11872%24155%40DEFAULT'
}

In [61]:
month_filter_clothes_uni = {
    '0~3개월': '223747%23attr_11872%24150%40DEFAULT',
    '3~6개월': '223747%23attr_11872%24151%40DEFAULT',
    '6~9개월': '223747%23attr_11872%24152%40DEFAULT',
    '9~12개월': '223747%23attr_11872%24153%40DEFAULT',
    '12~18개월': '223747%23attr_11872%24154%40DEFAULT',
    '18~24개월': '223747%23attr_11872%24155%40DEFAULT'
}

In [62]:
monthList_clothes = ['0~3개월', '3~6개월', '6~9개월', '9~12개월', '12~18개월', '18~24개월']

In [63]:
params_clothes_girl = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "223745",
    "rating": "0",
}

In [64]:
params_clothes_boy = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "223746",
    "rating": "0",
}

In [65]:
params_clothes_uni = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "223747",
    "rating": "0",
}

In [66]:
seed_clothes_girl = 'http://www.coupang.com/np/categories/223845'
seed_clothes_boy = 'http://www.coupang.com/np/categories/223846'
seed_clothes_uni = 'http://www.coupang.com/np/categories/223847'

In [67]:
result_clothes_girl = {
    monthList_clothes[0]: None,
    monthList_clothes[1]: None,
    monthList_clothes[2]: None,
    monthList_clothes[3]: None,
    monthList_clothes[4]: None,
    monthList_clothes[5]: None
}

for i in range(6):
    currentpage = 1
    params_clothes_girl['filter'] = month_filter_clothes_girl[monthList_clothes[i]]
    temp = []
    
    while True:
        params_clothes_girl['page'] = str(currentpage)
        print(params_clothes_girl['filter'])
        if len(getItemNum(seed_clothes_girl, params_clothes_girl)) == 0:
            break
        print(params_clothes_girl['page'])
        temp.extend(getURL(seed_clothes_girl, params_clothes_girl, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_clothes_girl[monthList_clothes[i]] = temp

223745%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24150%40DEFAULT&component=223745&rating=0
1
223745%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24150%40DEFAULT&component=223745&rating=0
2
223745%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24150%40DEFAULT&component=223745&rating=0
3
223745%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categ

9
223745%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24151%40DEFAULT&component=223745&rating=0
10
223745%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24151%40DEFAULT&component=223745&rating=0
11
223745%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24151%40DEFAULT&component=223745&rating=0
12
223745%23attr_11872%24151%40DEFAULT
http://www.coupang.com/

223745%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24153%40DEFAULT&component=223745&rating=0
1
223745%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24153%40DEFAULT&component=223745&rating=0
2
223745%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24153%40DEFAULT&component=223745&rating=0
3
223745%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categ

9
223745%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24154%40DEFAULT&component=223745&rating=0
10
223745%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24154%40DEFAULT&component=223745&rating=0
11
223745%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24154%40DEFAULT&component=223745&rating=0
12
223745%23attr_11872%24154%40DEFAULT
http://www.coupang.com/

In [68]:
for (k, v) in result_clothes_girl.items():
    print(k, len(v))

0~3개월 1020
3~6개월 1020
6~9개월 1020
9~12개월 1020
12~18개월 1020
18~24개월 1020


In [69]:
result_clothes_boy = {
    monthList_clothes[0]: None,
    monthList_clothes[1]: None,
    monthList_clothes[2]: None,
    monthList_clothes[3]: None,
    monthList_clothes[4]: None,
    monthList_clothes[5]: None
}

for i in range(6):
    currentpage = 1
    params_clothes_boy['filter'] = month_filter_clothes_boy[monthList_clothes[i]]
    temp = []
    
    while True:
        params_clothes_boy['page'] = str(currentpage)
        print(params_clothes_boy['filter'])
        if len(getItemNum(seed_clothes_boy, params_clothes_boy)) == 0:
            break
        print(params_clothes_boy['page'])
        temp.extend(getURL(seed_clothes_boy, params_clothes_boy, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_clothes_boy[monthList_clothes[i]] = temp

223746%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24150%40DEFAULT&component=223746&rating=0
1
223746%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24150%40DEFAULT&component=223746&rating=0
2
223746%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24150%40DEFAULT&component=223746&rating=0
3
223746%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categ

9
223746%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24151%40DEFAULT&component=223746&rating=0
10
223746%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24151%40DEFAULT&component=223746&rating=0
11
223746%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24151%40DEFAULT&component=223746&rating=0
12
223746%23attr_11872%24151%40DEFAULT
http://www.coupang.com/

223746%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24153%40DEFAULT&component=223746&rating=0
1
223746%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24153%40DEFAULT&component=223746&rating=0
2
223746%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24153%40DEFAULT&component=223746&rating=0
3
223746%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categ

9
223746%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24154%40DEFAULT&component=223746&rating=0
10
223746%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24154%40DEFAULT&component=223746&rating=0
11
223746%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24154%40DEFAULT&component=223746&rating=0
12
223746%23attr_11872%24154%40DEFAULT
http://www.coupang.com/

In [70]:
for (k, v) in result_clothes_boy.items():
    print(k, len(v))

0~3개월 1020
3~6개월 1020
6~9개월 1020
9~12개월 1020
12~18개월 1020
18~24개월 1020


In [71]:
result_clothes_uni = {
    monthList_clothes[0]: None,
    monthList_clothes[1]: None,
    monthList_clothes[2]: None,
    monthList_clothes[3]: None,
    monthList_clothes[4]: None,
    monthList_clothes[5]: None
}

for i in range(6):
    currentpage = 1
    params_clothes_uni['filter'] = month_filter_clothes_uni[monthList_clothes[i]]
    temp = []
    
    while True:
        params_clothes_uni['page'] = str(currentpage)
        print(params_clothes_uni['filter'])
        if len(getItemNum(seed_clothes_uni, params_clothes_uni)) == 0:
            break
        print(params_clothes_uni['page'])
        temp.extend(getURL(seed_clothes_uni, params_clothes_uni, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_clothes_uni[monthList_clothes[i]] = temp

223747%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24150%40DEFAULT&component=223747&rating=0
1
223747%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24150%40DEFAULT&component=223747&rating=0
2
223747%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24150%40DEFAULT&component=223747&rating=0
3
223747%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categ

9
223747%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24151%40DEFAULT&component=223747&rating=0
10
223747%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24151%40DEFAULT&component=223747&rating=0
11
223747%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24151%40DEFAULT&component=223747&rating=0
12
223747%23attr_11872%24151%40DEFAULT
http://www.coupang.com/

223747%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24153%40DEFAULT&component=223747&rating=0
1
223747%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24153%40DEFAULT&component=223747&rating=0
2
223747%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24153%40DEFAULT&component=223747&rating=0
3
223747%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categ

9
223747%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24154%40DEFAULT&component=223747&rating=0
10
223747%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24154%40DEFAULT&component=223747&rating=0
11
223747%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24154%40DEFAULT&component=223747&rating=0
12
223747%23attr_11872%24154%40DEFAULT
http://www.coupang.com/

In [72]:
for (k, v) in result_clothes_uni.items():
    print(k, len(v))

0~3개월 1020
3~6개월 1020
6~9개월 1020
9~12개월 1020
12~18개월 1020
18~24개월 1020


In [79]:
clothesList = []

In [77]:
def getInfo_clothes(url, monthInfo, gender, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_clothes(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and len(dom.select('span.total-price strong')) > 0:
        name = dom.select('h2.prod-buy-header__title')[0].get_text()
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        url = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        clothesList.append(tuple([name, gender, price, url, img, monthInfo]))
    
    return None

In [80]:
for i in range(6):
    for url in result_clothes_girl[monthList_clothes[i]]:
        getInfo_clothes(url, monthList_clothes[i], gender_list[0])
    for url in result_clothes_boy[monthList_clothes[i]]:
        getInfo_clothes(url, monthList_clothes[i], gender_list[1])
    for url in result_clothes_uni[monthList_clothes[i]]:
        getInfo_clothes(url, monthList_clothes[i], gender_list[2])

In [81]:
len(clothesList)

18088